In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle

#IMPORTING H5PY
###############################################################
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
import h5py #also in sbatch add: export HDF5_USE_FILE_LOCKING=FALSE
###############################################################

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Variable_Calculation/LagrangianArrays'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data1,parcel1] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'

In [ ]:
#########################################

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
##############################################
#DATA INITIALIZATION and LOADING FUNCTIONS

In [ ]:
#INITIALIZE DATA FUNCTION
###############################################################
def initiate_array(out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = len(data['time'])  # Number of timesteps
    if p_size is None:
        p_size = len(parcel['xh'])  # Number of vertical levels

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g', 'A_c']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )

In [ ]:
#INITIALIZE DATA FUNCTION
def InitiateArray_Job(data,out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = len(data['time'])  # Number of timesteps
    if p_size is None:
        p_size = len(parcel1['xh'])  # Number of vertical levels

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g', 'A_c']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id,num_jobs):
    total_elements=len(data1['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
    #     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    #     if job_id==0: job_id=1
        
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    return start_job,end_job,index_adjust
# job_id=1;num_jobs=60
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# data=data1.isel(time=slice(start_job,end_job))
# parcel=parcel1.isel(time=slice(start_job,end_job))

In [ ]:
#################################################################################################################################################

In [ ]:
###############################################################
#FUNCTIONS

In [ ]:
def GetZYX(parcel):
    #Lagrangian Position Arrays
    ##############
    def grid_location(z,y,x):
        zf=data1['zf'].values*1000; which_zh=np.clip(np.searchsorted(zf,z)-1,0,None).astype(np.uint16)
        #which_zh=np.where(which_zh == -1, 0, which_zh) 
        
        yf=data1['yf'].values*1000; which_yh=np.clip(np.searchsorted(yf,y)-1,0,None).astype(np.uint16) 
        #which_yh=np.where(which_yh == -1, 0, which_yh) 
        
        xf=data1['xf'].values*1000; which_xh=np.clip(np.searchsorted(xf,x)-1,0,None).astype(np.uint16)
        #which_xh=np.where(which_xh == -1, 0, which_xh) 
        
        return which_zh,which_yh,which_xh
    
    print('Creating Lagrangian X,Y,Z Binary Arrays')
    x=parcel['x'].data;y=parcel['y'].data;z=parcel['z'].data
    Z,Y,X=grid_location(z,y,x)
    
    return z,x,Z,Y,X
# [z,x,Z,Y,X]=GetZYX(parcel)

def GetUW(parcel):
    #GETTING u and w for Lagrangian_Tracking later
    u=parcel['u'].data
    w=parcel['w'].data
    return u,w
# [u,w]=GetUW(parcel)

In [ ]:
def call_variable(data,varname):
    var_data=data[varname].data
    return var_data
# def MakeLagrangianArray(data,varnames,Z,Y,X): #SLOW FOR LOOP VERSION
#     # Initialize dictionaries
#     var_data_dict = {varname: call_variable(data,varname) for varname in varnames}
#     VAR = {varname: np.zeros_like(Z, dtype='float32') for varname in varnames}

#     Nt = len(data['time'])
#     Np = len(parcel1['xh'])
#     for p in np.arange(Np):
#         # if np.mod(p, 1e6) == 0: 
#         #     print(f"{p}/{len(parcel1['xh'])}")

#         # Get Indices
#         zs = Z[:, p]
#         ys = Y[:, p]
#         xs = X[:, p]
#         ts = np.arange(Nt)

#         # Loop over all variables and fill the respective VAR array
#         for varname, var_data in var_data_dict.items():
#             VAR[varname][:, p] = var_data[ts, zs, ys, xs]

#     # Return all the arrays in a list
#     return [VAR[varname] for varname in varnames]

def MakeLagrangianArray(data,varnames,Z,Y,X): #FAST ADVANCED INDEXING VERSION
    print('Making Variables Lagrangian Array')
    # Initialize dictionaries
    var_data_dict = {varname: call_variable(data,varname) for varname in varnames}
    VAR = {varname: np.zeros_like(Z, dtype='float32') for varname in varnames}

    print('Running')
    Nt = len(data['time'])
    # Get Indices
    z_idx = Z
    y_idx = Y
    x_idx = X
    ts = np.arange(Nt)[:, None]
    t_idx = np.broadcast_to(ts, Z.shape)

    for varname, var_data in var_data_dict.items():
        VAR[varname] = var_data[t_idx, z_idx, y_idx, x_idx]

    return [VAR[varname] for varname in varnames]
    # return VAR

In [ ]:
import dask.array as da
def GetVariables(data,Z,Y,X):
    print('Making W, QC, and QI Lagrangian Array')
    varnames=['winterp','qc','qi']
    [W,QC,QI]=MakeLagrangianArray(data,varnames,Z,Y,X); 
    
    print('Making QC+QI Lagrangian Array')
    Nt=len(data['time'])
    QC = da.from_array(QC, chunks=(Nt, 'auto'))
    QI = da.from_array(QI, chunks=(Nt, 'auto'))
    QCQI=QC+QI
    QCQI=QCQI.compute();
    
    return W,QC,QCQI
# [W,QC,QCQI]=GetVariables(data,Z,Y,X)

In [ ]:
def GetBinaryArrays(data,W,QCQI):
    #Create Set Thresholds and Create Binary Arrays
    w_thresh1=0.1
    w_thresh2=0.5
    qcqi_thresh1=1e-6
    qcqi_thresh2=1e-6
    
    
    #Apply Thresholds 
    Nt=len(data['time'])
    W = da.from_array(W, chunks=(Nt, 'auto'))
    QCQI = da.from_array(QCQI, chunks=(Nt, 'auto'))

    print('Making Lagrangian Binary Arrays')
    print('Making A_g')
    A_g = np.where((W >= w_thresh1) & (QCQI < qcqi_thresh1), True, False)
    print('Making A_c')
    A_c = np.where((W >= w_thresh2) & (QCQI >= qcqi_thresh2), True, False)
    A_g=A_g.compute(); A_c=A_c.compute()
    print('done')
    
    return A_g,A_c
# [A_g,A_c]=GetBinaryArrays(data,W,QCQI)

In [ ]:
def SaveData(data,u,w,z,x,Z,Y,X,W,QCQI,A_g,A_c):
    # Saving Data
    ##############
    print('Saving Data\n')
    import h5py
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/SBATCH/job_out/'
    out_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}'
    out_file+=f'_{job_id}.h5'
    
    vars=['u','w','z','x','Z','Y','X','W','QCQI','A_g','A_c']
    InitiateArray_Job(data,out_file,vars,t_chunk_size=1,p_chunk_size=1_000_000)
    with h5py.File(out_file, 'a') as f: 
        for var in vars:
            f[var][:] = locals()[var]
# SaveData(data,u,w,z,x,Z,Y,X,W,QCQI,A_g,A_c)            

In [ ]:
###############################################################
#RUNNING

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if res == "1km" and Np_str=='1e6':
    num_jobs=60 #1M parcels
    num_slurm_jobs=10
elif res == "1km" and Np_str=='50e6':
    num_jobs=400 #50M parcels
    num_slurm_jobs=60
elif res == "250m" and Np_str=='50e6':
    num_jobs=600 #50M parcels
    num_slurm_jobs=150
##############################

In [ ]:
################################
#RUN SETUP
###############################

In [ ]:
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=True) #if ISRUN is False, then will not run using slurm_job_array
print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")

job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 == 0: print(f'current job_id = {job_id}/{num_jobs}')
    [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

    #SLICING DATA
    [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
    data=data1.isel(time=slice(start_job,end_job))
    parcel=parcel1.isel(time=slice(start_job,end_job))

    #GETTING POSITION VARIABLES
    [z,x,Z,Y,X]=GetZYX(parcel)
    [u,w]=GetUW(parcel)

    #MAKING LAGRANGIAN ARRAYS (for threshold)
    [W,QC,QCQI]=GetVariables(data,Z,Y,X)
    #MAKING BINARY ARRAYS
    [A_g,A_c]=GetBinaryArrays(data,W,QCQI)

    #SAVING
    SaveData(data,u,w,z,x,Z,Y,X,W,QCQI,A_g,A_c)

    #check_memory(globals())
    del u,w,z,x,Z,Y,X,W,QCQI,A_g,A_c 

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
# recombine=True

In [ ]:
# #OLD
# def Recombine(num_jobs):
#     dir2=dir+'Project_Algorithms/Lagrangian_Arrays/SBATCH/job_out/'
#     dir3=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
#     out_file=dir3+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'

#     print('initializing array')
#     vars=['u','w','z','Z','Y','X','W','QCQI','A_g','A_c']
#     InitiateArray_Job(out_file,vars,t_chunk_size=100,p_chunk_size=500_000)

#     print('recombining')
#     with h5py.File(out_file, 'r+') as f_out:
#         for job_id in np.arange(1,num_jobs+1):
#             if job_id % 5: print(f"job_id = {job_id}")
#             [a,b] = get_job_range(job_id,num_jobs)
    
#             in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}_{job_id}.h5' 
#             with h5py.File(in_file, 'r') as f_in: 
#                 for var in vars:
#                     f_out[var][a:b]=f_in[var][:]

In [ ]:
def Recombine_Dask():
    #DASK-ENABLED
    import glob
    import re
    def recombine_func(in_files,out_file):
        # matching_files = sorted(glob.glob(in_files))
        matching_files = sorted(
        glob.glob(in_files),
        key=lambda f: int(re.search(r'_(\d+)\.h5$', f).group(1))
    )
        
        print(f'recombining {len(matching_files)} files')
        out=xr.open_mfdataset(matching_files,engine='h5netcdf',concat_dim='phony_dim_0',combine='nested',phony_dims='sort')
        from dask.diagnostics import ProgressBar
        with ProgressBar():
            out.to_netcdf(out_file, engine='h5netcdf')
        

    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/SBATCH/job_out/'
    dir3=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    in_files = dir2 + f'lagrangian_binary_array_{res}_{t_res}_{Np_str}_*.h5'
    out_file=dir3+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'  #TESTING
    
    recombine_func(in_files,out_file)

In [ ]:
if recombine==True:
    # Recombine(num_jobs=num_jobs)
    Recombine_Dask()

In [ ]:
#########################################
# Reading Back Data Later

In [ ]:
# # Reading Back Data Later
# ##############
# def make_data_dict(in_file,var_names,read_type,start_job,end_job):
#     if read_type=='h5py':
#         with h5py.File(in_file, 'r') as f:
#            data_dict = {var_name: f[var_name][:,start_job:end_job] for var_name in var_names}
            
#     elif read_type=='xarray':
#         in_data = xr.open_dataset(
#             in_file,
#             engine='h5netcdf',
#             phony_dims='sort',
#             chunks={'phony_dim_0': 100, 'phony_dim_1': 1_000_000} 
#         )
#         data_dict = {k: in_data[k][:,start_job:end_job].compute().data for k in var_names}
#     return data_dict

# # read_type='xarray'
# read_type='h5py'

In [ ]:
# import h5py
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'

# var_names = ['A_g', 'A_c', 'Z', 'Y', 'X','W']
# data_dict = make_data_dict(var_names,read_type)
# A_g, A_c, Z, Y, X, W = (data_dict[k] for k in var_names)

# # #Making Time Matrix
# Nt=len(data['time'])
# T = np.broadcast_to(np.arange(Nt)[:, None], A_c.shape)  # shape: (Nt, p)

# check_memory(globals())

In [ ]:
# #TESTING ARE RESULTS CORRECT
# job_id=50;num_jobs=60
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# # print(start_job,end_job)
# # start_job,end_job=0,3

# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# with h5py.File(dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5', 'r') as f:
#     # Load the dataset by its name
#     parcel_z = f['z'][start_job:end_job]
#     Z = f['Z'][start_job:end_job]
#     Y = f['Y'][start_job:end_job]
#     X = f['X'][start_job:end_job]

#     A_c = f['A_c'][start_job:end_job]
#     A_g = f['A_g'][start_job:end_job]
#     W = f['W'][start_job:end_job]
#     QCQI = f['QCQI'][start_job:end_job]

# #TESTING

# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
# open_file = dir2 + f'lagrangian_binary_array_{res}_{t_res}_{Np_str}_{job_id}.h5'
# with h5py.File(open_file, 'r') as f:
#     key_list=list(f.keys())
#     for key in f:
#         # print(key)
#         globals()[key]=f[key][:]
# data2=data1.isel(time=slice(start_job,end_job))

In [ ]:
# #TESTING ARE RESULTS CORRECT
# t,p=0,10000
# def test(t,p,VAR):
#     z=Z[t,p];y=Y[t,p];x=X[t,p]
#     out1=data2['winterp'].isel(time=t,zh=z,yh=y,xh=x).data
#     out2=data2['qc'].isel(time=t,zh=z,yh=y,xh=x).data
#     out3=data2['qi'].isel(time=t,zh=z,yh=y,xh=x).data
#     out4=out2+out3

#     if key=='A_c':
#         out4=np.where((out1>=0.5)&(out4>=1e-6),True,False)
#     if key=='A_g':
#         out4=np.where((out1>=0.1)&(out4<1e-6),True,False)
#     print(VAR[t,p],out4)
# for key in ['A_c','A_g']:
#     test(t,p,globals()[key])